In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210619.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice]
0,15/06/2021 20:22:01,A Generation (1955),Stray Dog (1949),Shiri (1999),Drug War (2012),The Long Riders (1980),Guns Akimbo (2019),THE GLOBAL SOUTH,MOVIE ABOUT A PARTY,BOXING IS THE GREATEST MOVIE SPORT,BOLLYWOOD,[Default to Nicolas Cage],LET'S PUT ON A SHOW!!,"LOW EXPECTATIONS, DECENT MOVIE"
1,15/06/2021 20:38:32,A Generation (1955),Shiri (1999),Guns Akimbo (2019),Stray Dog (1949),Drug War (2012),The Long Riders (1980),BOLLYWOOD,MOVIE ABOUT A PARTY,[Default to Nicolas Cage],LET'S PUT ON A SHOW!!,"LOW EXPECTATIONS, DECENT MOVIE",THE GLOBAL SOUTH,BOXING IS THE GREATEST MOVIE SPORT
2,17/06/2021 04:58:40,Drug War (2012),The Long Riders (1980),Stray Dog (1949),Guns Akimbo (2019),Shiri (1999),A Generation (1955),THE GLOBAL SOUTH,BOLLYWOOD,[Default to Nicolas Cage],MOVIE ABOUT A PARTY,"LOW EXPECTATIONS, DECENT MOVIE",BOXING IS THE GREATEST MOVIE SPORT,LET'S PUT ON A SHOW!!
3,17/06/2021 05:00:18,Drug War (2012),Guns Akimbo (2019),A Generation (1955),Stray Dog (1949),Shiri (1999),The Long Riders (1980),LET'S PUT ON A SHOW!!,"LOW EXPECTATIONS, DECENT MOVIE",BOXING IS THE GREATEST MOVIE SPORT,BOLLYWOOD,MOVIE ABOUT A PARTY,THE GLOBAL SOUTH,[Default to Nicolas Cage]
4,17/06/2021 14:18:29,Stray Dog (1949),A Generation (1955),The Long Riders (1980),Drug War (2012),Guns Akimbo (2019),Shiri (1999),BOXING IS THE GREATEST MOVIE SPORT,"LOW EXPECTATIONS, DECENT MOVIE",LET'S PUT ON A SHOW!!,MOVIE ABOUT A PARTY,[Default to Nicolas Cage],THE GLOBAL SOUTH,BOLLYWOOD


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['A Generation (1955)', 'Stray Dog (1949)', 'Shiri (1999)',
       'Drug War (2012)', 'The Long Riders (1980)', 'Guns Akimbo (2019)'],
      dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'A Generation (1955)': <Candidate('A Generation (1955)')>,
 'Stray Dog (1949)': <Candidate('Stray Dog (1949)')>,
 'Shiri (1999)': <Candidate('Shiri (1999)')>,
 'Drug War (2012)': <Candidate('Drug War (2012)')>,
 'The Long Riders (1980)': <Candidate('The Long Riders (1980)')>,
 'Guns Akimbo (2019)': <Candidate('Guns Akimbo (2019)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(A Generation (1955), Stray Dog (1949), Shiri (1999), Drug War (2012), The Long Riders (1980), Guns Akimbo (2019))>,
 <Ballot(A Generation (1955), Shiri (1999), Guns Akimbo (2019), Stray Dog (1949), Drug War (2012), The Long Riders (1980))>,
 <Ballot(Drug War (2012), The Long Riders (1980), Stray Dog (1949), Guns Akimbo (2019), Shiri (1999), A Generation (1955))>,
 <Ballot(Drug War (2012), Guns Akimbo (2019), A Generation (1955), Stray Dog (1949), Shiri (1999), The Long Riders (1980))>,
 <Ballot(Stray Dog (1949), A Generation (1955), The Long Riders (1980), Drug War (2012), Guns Akimbo (2019), Shiri (1999))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                 Votes  Status
----------------------  -------  --------
A Generation (1955)           2  Hopeful
Drug War (2012)               2  Hopeful
Stray Dog (1949)              1  Rejected
Guns Akimbo (2019)            0  Rejected
Shiri (1999)                  0  Rejected
The Long Riders (1980)        0  Rejected

FINAL RESULT
Candidate                 Votes  Status
----------------------  -------  --------
A Generation (1955)           3  Elected
Drug War (2012)               2  Rejected
Stray Dog (1949)              0  Rejected
Guns Akimbo (2019)            0  Rejected
Shiri (1999)                  0  Rejected
The Long Riders (1980)        0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                             Votes  Status
----------------------------------  -------  --------
THE GLOBAL SOUTH                          2  Hopeful
BOLLYWOOD                                 1  Hopeful
BOXING IS THE GREATEST MOVIE SPORT        1  Hopeful
LET'S PUT ON A SHOW!!                     1  Rejected
MOVIE ABOUT A PARTY                       0  Rejected
LOW EXPECTATIONS, DECENT MOVIE            0  Rejected
[Default to Nicolas Cage]                 0  Rejected

ROUND 2
Candidate                             Votes  Status
----------------------------------  -------  --------
THE GLOBAL SOUTH                          2  Hopeful
BOXING IS THE GREATEST MOVIE SPORT        2  Hopeful
BOLLYWOOD                                 1  Rejected
LET'S PUT ON A SHOW!!                     0  Rejected
MOVIE ABOUT A PARTY                       0  Rejected
LOW EXPECTATIONS, DECENT MOVIE            0  Rejected
[Default to Nicolas Cage]                 0  Rejected

FINAL RESULT
Candid